### Building CNN Architectures with PyTorch

## Key Steps in Building CNN Architectures with PyTorch

1. **Define a Model**
    - Create a class that inherits from `torch.nn.Module`.
    - Use layers such as convolutional, pooling, and fully connected layers to build your architecture.

2. **Build CNN Layers**
    - Use modules like `nn.Conv2d`, `nn.MaxPool2d`, and `nn.Linear` to construct the network.
    - Stack layers to increase model capacity and capture complex patterns.

3. **Forward Pass**
    - Implement the `forward()` method to define how input data flows through the layers to produce output.
    - Apply activation functions (e.g., ReLU) and flatten outputs as needed for fully connected layers.

4. **Model Summary**
    - Inspect the structure and learnable parameters using `print(model)` or libraries like `torchsummary`.
    - Understand the number of parameters and the shape of outputs at each layer.

---

## Training and Evaluating CNNs in PyTorch

- **Training**
  - Perform forward and backward passes.
  - Calculate loss using functions like `nn.CrossEntropyLoss`.
  - Update weights using optimizers such as `torch.optim.Adam` or `torch.optim.SGD`.
  - Iterate over epochs and batches to improve model performance.

- **Evaluation**
  - Test the model on unseen (validation/test) data.
  - Compute metrics such as accuracy, loss, precision, and recall.
  - Use `model.eval()` and disable gradient computation with `torch.no_grad()` during evaluation.

---

## Experimenting with CNN Model Design and Hyperparameter Tuning

### Areas for Experimentation

- **Layer Depth**
  - Add or remove convolutional and pooling layers to observe their impact on learning and generalization.

- **Filter Size**
  - Experiment with different kernel sizes (e.g., 3x3, 5x5) to capture features at various scales.

- **Learning Rate**
  - Adjust the learning rate to improve convergence speed and model accuracy.
  - Consider using learning rate schedulers for dynamic adjustment during training.

- **Batch Size**
  - Tune batch size for optimal memory usage and training stability.

- **Regularization**
  - Apply dropout (`nn.Dropout`) or weight decay to prevent overfitting.

- **Data Augmentation**
  - Use transforms (e.g., random crop, flip, rotation) to increase dataset diversity and robustness.

---

> **Tip:** Systematic experimentation and careful tracking of results are key to finding the best model configuration.

In [13]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

Define transformation

In [14]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

# load cifar-10 dataset
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    transform=transform,
)
test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    transform=transform,
)



create data loaders

In [15]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training Data Size {len(train_dataset)}")
print(f"Test Data Size {len(test_dataset)}")    


Training Data Size 50000
Test Data Size 10000


Define CNN model

In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = CNN()
print(model)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


define loss function and optimse

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            # zero graident
            optimizer.zero_grad()

            # forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # backward pass and optimise
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1} loss: {running_loss/len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)

# evaluation loop
def evaluate_model(model, test_loader):
    model.eval()
    correct= 0
    total= 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct/total}%")

evaluate_model(model, test_loader)  


Epoch 1 loss: 1.6466952153788808
Epoch 2 loss: 1.345947896580562
Epoch 3 loss: 1.2257301012420898
Epoch 4 loss: 1.15537971410605
Epoch 5 loss: 1.0939511977650624
Epoch 6 loss: 1.038463511811498
Epoch 7 loss: 0.9976639953415717
Epoch 8 loss: 0.9562068307948539
Epoch 9 loss: 0.9150536965836039
Epoch 10 loss: 0.880431536289737
Accuracy: 62.82%
